In [ ]:
import pandas as pd

In [4]:
collagens = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\collagens_proteins\\predictions_vs_real_collagens_proteins.csv")
# add column with the protein type
collagens["protein_type"] = "collagens_proteins"

globular = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\globular_proteins\\predictions_vs_real_globular_proteins.csv")
# add column with the protein type
globular["protein_type"] = "globular_proteins"

transmembrane = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\transmembrane_residues\\predictions_vs_real_transmembrane_residues.csv")
# add column with the protein type
transmembrane["protein_type"] = "transmembrane_residues"

# concatenate the three dataframes
my_predictions_all_proteins = pd.concat([collagens, globular, transmembrane])
# change column name from "reality" to "pathogenicity"
my_predictions_all_proteins.rename(columns={"reality": "pathogenicity"}, inplace=True)

my_predictions_all_proteins

,gene,variant,prediction,pathogenicity,protein_type
0,COL11A1,G1249V,1,1,collagens_proteins
1,COL11A1,G1516V,1,1,collagens_proteins
2,COL11A1,R1733C,1,1,collagens_proteins
3,COL11A1,G1800C,1,1,collagens_proteins
4,COL11A1,R1076C,1,1,collagens_proteins
...,...,...,...,...,...
504,FITM2,K39R,1,0,transmembrane_residues
505,ADCY1,A83V,0,0,transmembrane_residues
506,S1PR2,V286A,1,0,transmembrane_residues
507,SYNE4,V368M,0,0,transmembrane_residues


In [5]:
tools = ['alphamissense']

In [ ]:
# Create merge, which is the my_prediction dataframe with the predictions of the other tools
merged = my_predictions_all_proteins
for tool in tools:
    combine_tool = pd.DataFrame()
    # combine the three dataframes of the tool by variant and gene
    for protein_type in ['collagens_proteins', 'globular_proteins', 'transmembrane_residues']:
        tool_df = pd.read_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\protein_type_predictors\\alphamissense_on_my_data_predictions_{protein_type}.csv")
        combine_tool = pd.concat([combine_tool, tool_df])
    # Delete prediction and reality columns
    combine_tool.drop(columns=["prediction", "reality"], inplace=True)
    # combine with the merged dataframe
    merged = pd.merge(merged, combine_tool, on=["variant", "gene"], how="left")
    # delete rows with Nan in am_class
    merged = merged[merged["am_class"].notna()]
    # delete rows where am_class is ambiguous
    merged = merged[merged["am_class"] != "ambiguous"]
merged

In [22]:

# convert the column "am_class" to int
merged["am_class"] = merged["am_class"].astype(int)

merged


,gene,variant,prediction,pathogenicity,protein_type,Unnamed: 0,uniprot_id,am_pathogenicity,am_class
0,COL11A1,G1249V,1,1,collagens_proteins,0.0,P12107,0.9505,1
1,COL11A1,G1516V,1,1,collagens_proteins,1.0,P12107,0.9748,1
2,COL11A1,R1733C,1,1,collagens_proteins,2.0,P12107,0.7065,1
3,COL11A1,G1800C,1,1,collagens_proteins,3.0,P12107,0.9094,1
4,COL11A1,R1076C,1,1,collagens_proteins,4.0,P12107,0.8094,1
...,...,...,...,...,...,...,...,...,...
3911,FITM2,K39R,1,0,transmembrane_residues,504.0,Q8N6M3,0.1527,0
3912,ADCY1,A83V,0,0,transmembrane_residues,505.0,Q08828,0.0937,0
3913,S1PR2,V286A,1,0,transmembrane_residues,506.0,O95136,0.2993,0
3914,SYNE4,V368M,0,0,transmembrane_residues,507.0,Q8N205,0.1570,0


In [23]:
# print how many variants there are from each gene and each pathogenicity
for gene in merged["gene"].unique():
    print(f"Gene: {gene}")
    print(merged[merged["gene"] == gene]["pathogenicity"].value_counts())

Gene: COL11A1
pathogenicity
1    33
0     8
Name: count, dtype: int64
Gene: COL4A3
pathogenicity
1    112
0     18
Name: count, dtype: int64
Gene: COL9A3
pathogenicity
0    15
1     5
Name: count, dtype: int64
Gene: COL11A2
pathogenicity
1    17
0    10
Name: count, dtype: int64
Gene: COL4A5
pathogenicity
1    383
0     13
Name: count, dtype: int64
Gene: COL2A1
pathogenicity
1    281
0     10
Name: count, dtype: int64
Gene: COL4A4
pathogenicity
1    88
0    21
Name: count, dtype: int64
Gene: COL9A1
pathogenicity
0    5
1    1
Name: count, dtype: int64
Gene: COL9A2
pathogenicity
0    14
Name: count, dtype: int64
Gene: COL4A6
pathogenicity
0    1
Name: count, dtype: int64
Gene: POLR1D
pathogenicity
1    9
Name: count, dtype: int64
Gene: NDP
pathogenicity
1    80
0     1
Name: count, dtype: int64
Gene: USH1G
pathogenicity
1    8
0    2
Name: count, dtype: int64
Gene: OSBPL2
pathogenicity
0    2
1    1
Name: count, dtype: int64
Gene: ESRRB
pathogenicity
1    14
0     4
Name: count, dtype: 

In [24]:
## Calculate the MCC for the predictions, according to the real pathogenicity (0 or 1)
from sklearn.metrics import matthews_corrcoef

In [25]:
# separate the merged dataframe according to gene
genes = merged["gene"].unique()
print(f"Number of genes: {len(genes)}")

Number of genes: 169


In [31]:
# Calculate MCC for each protein specific predictor for AM

# Build dictionary with gene names as keys.
mccs = {protein_type: 0 for protein_type in ["collagens_proteins", "globular_proteins", "transmembrane_residues"]}

for tool in tools:
    if tool == "alphamissense":
        tool = "am"
    for protein_type in ["collagens_proteins", "globular_proteins", "transmembrane_residues"]:
        protein_type_df = merged[merged["protein_type"] == protein_type]
        # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
        # Create binary predictions based on the 0.5 threshold
        
        protein_type_df['pathogenicity'] = protein_type_df['pathogenicity'].astype(int)
        
        # Get TP, TN, FP, FN
        TP = len(protein_type_df[(protein_type_df['pathogenicity'] == 1) & (protein_type_df[f"{tool}_class"] == 1)])
        TN = len(protein_type_df[(protein_type_df['pathogenicity'] == 0) & (protein_type_df[f"{tool}_class"] == 0)])
        FP = len(protein_type_df[(protein_type_df['pathogenicity'] == 0) & (protein_type_df[f"{tool}_class"] == 1)])
        FN = len(protein_type_df[(protein_type_df['pathogenicity'] == 1) & (protein_type_df[f"{tool}_class"] == 0)])
        print(f"{tool} --{protein_type} TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
        
        # Calculate MCC
        mcc = matthews_corrcoef(protein_type_df['pathogenicity'], protein_type_df[f"{tool}_class"])
        
        # Get gene name for the use for the dictionary
        protein_type = protein_type_df['protein_type'].unique()[0]
        
        # Append mcc to dictionary
        mccs[protein_type] = mcc
    
    print(f"MCCs of {tool} predictions for each gene:")
    for protein_type in mccs:
        print(f"{protein_type}: {mccs[protein_type]}")
        
# get MCC for my predictions
mccs = {protein_type: 0 for protein_type in ["collagens_proteins", "globular_proteins", "transmembrane_residues"]}

for protein_type in ["collagens_proteins", "globular_proteins", "transmembrane_residues"]:
    protein_type_df = merged[merged["protein_type"] == protein_type]
    
    protein_type_df['pathogenicity'] = protein_type_df['pathogenicity'].astype(int)
    protein_type_df['prediction'] = protein_type_df['prediction'].astype(int)
    
    # Get TP, TN, FP, FN
    TP = len(protein_type_df[(protein_type_df['pathogenicity'] == 1) & (protein_type_df['prediction'] == 1)])
    TN = len(protein_type_df[(protein_type_df['pathogenicity'] == 0) & (protein_type_df['prediction'] == 0)])
    FP = len(protein_type_df[(protein_type_df['pathogenicity'] == 0) & (protein_type_df['prediction'] == 1)])
    FN = len(protein_type_df[(protein_type_df['pathogenicity'] == 1) & (protein_type_df['prediction'] == 0)])
    
    print(f"my tool: {protein_type} TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
    
    # Calculate MCC
    mcc = matthews_corrcoef(protein_type_df['pathogenicity'], protein_type_df['prediction'])
    
    # Get gene name for the use for the dictionary
    protein_type = protein_type_df['protein_type'].unique()[0]
    
    # Append mcc to dictionary
    mccs[protein_type] = mcc

print(f"MCCs of predictions for each gene:")
for protein_type in mccs:
    print(f"{protein_type}: {mccs[protein_type]}")


am --collagens_proteins TP: 831, TN: 111, FP: 4, FN: 89
am --globular_proteins TP: 1264, TN: 543, FP: 38, FN: 276
am --transmembrane_residues TP: 368, TN: 35, FP: 6, FN: 52
MCCs of am predictions for each gene:
collagens_proteins: 0.6912633795307253
globular_proteins: 0.691932045668408
transmembrane_residues: 0.5309506674270398
my tool: collagens_proteins TP: 832, TN: 95, FP: 20, FN: 88
my tool: globular_proteins TP: 1235, TN: 430, FP: 151, FN: 305
my tool: transmembrane_residues TP: 384, TN: 22, FP: 19, FN: 36
MCCs of predictions for each gene:
collagens_proteins: 0.6016985117592833
globular_proteins: 0.5079998102014048
transmembrane_residues: 0.3869923974611639


C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_9860\1735904402.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protein_type_df['pathogenicity'] = protein_type_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_9860\1735904402.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protein_type_df['pathogenicity'] = protein_type_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_9860\1735904402.py:14: SettingWithCopyWarning: 
A value is trying t

In [14]:
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, f1_score, log_loss, confusion_matrix
# store the results in a dictionary
results = {gene: 0 for gene in genes}

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    y_true = gene_df["pathogenicity"].astype(int)
    predicted_probabilities = gene_df["MutPred_score"]
    gene_df.loc[:, 'binary_prediction_mutpred'] = (gene_df['MutPred_score'] > 0.5).astype(int)
    predicted_labels = gene_df["binary_prediction_mutpred"]
    
    results_gene = {}
    # Calculate AUC-ROC
    roc_auc = roc_auc_score(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["roc_auc"] = roc_auc
    
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, predicted_probabilities)
    roc_auc = auc(fpr, tpr)
    # append to dictionary
    results_gene["auc"] = roc_auc
    
    # Calculate AUC-PR
    precision, recall, _ = precision_recall_curve(y_true, predicted_probabilities)
    pr_auc = auc(recall, precision)
    # append to dictionary
    results_gene["pr_auc"] = pr_auc
    
    # Calculate F1 Score
    f1 = f1_score(y_true, predicted_labels)
    # append to dictionary
    results_gene["f1"] = f1
    
    # Calculate Log Loss
    logloss = log_loss(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["logloss"] = logloss
    
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(y_true, predicted_labels)
    # append to dictionary
    results_gene["confusion_matrix"] = conf_matrix
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    # Append results to dictionary
    results[gene] = results_gene
    
## Print results
print("Results of MutPred2 predictions for each gene:")
for gene in results:
    print(f"{gene}: {results[gene]}")

KeyError: 'MutPred_score'

In [12]:
# present the confusion matrix for each gene
import matplotlib.pyplot as plt

for gene in results:
    # Plot confusion matrix
    plt.imshow(results[gene]["confusion_matrix"], cmap=plt.cm.Blues)
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.xticks([0, 1], ["Benign", "Pathogenic"])
    plt.yticks([0, 1], ["Benign", "Pathogenic"])
    plt.title(f"Confusion matrix for {gene}, MutPred2 predictions")
    plt.style.use("seaborn-white")
    # Add text annotations
    for i in range(2):
        for j in range(2):
            plt.text(j, i, results[gene]["confusion_matrix"][i, j], ha="center", va="center", color="black", backgroundcolor="white")
    plt.colorbar()
    plt.show()

NameError: name 'results' is not defined

In [13]:
# present the ROC curve for each gene
for gene in results:
    # Plot ROC curve
    plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {results[gene]['roc_auc']:.2f})")
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC curve for {gene}, MutPred2 predictions")
    plt.legend(loc="lower right")
    plt.show()

NameError: name 'results' is not defined